In [56]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import codecs

In [53]:
# 게시물 1개 수집하는 함수
def brunch_to_html(url):
    # 웹페이지 수집
    soup = BeautifulSoup(urlopen(url), 'html.parser')
    articles = soup.findAll(re.compile(r'(p|div|h[1-6]|blockquote)'),     {'class':re.compile(r'(wrap_item item_type_text|wrap_item item_type_bullet|wrap_img_float)')})
#     articles[9].text
    ## 제목 추출
    title = soup.find('h1', {'class':'cover_title'}).text
    ## 소제목 추출
    sub_title = soup.find('p', {'class':'cover_sub_title'}).text
    # 결과물의 body 생성
    body = '<h1>' + title + '</h1><br><h3>' + sub_title + '</h3><br>'
    # 본문에서 텍스트와 그림 링크를 추출하여 body 생성
    for line in articles:
        if (line.text != '\xa0'):# & (line.text != ''):
            if str(line).__contains__('div'):
                try:
                    img_link = re.search(r'http.*', line.find('img')['src']).group(0)
                except:
                    img_link = ''
                try:
                    caption = line.find('span', {'class':'text_caption'}).text
                except:
                    caption = ''
                body += '<center><img src="' + img_link + '"><br>' + caption + '</center><br>'
            elif str(line).__contains__('h2'):
                body += '<br><h2>' + line.text + '</h2><br>'
            elif str(line).__contains__('h3'):
                body += '<h3>' + line.text + '</h3>'
            elif str(line).__contains__('h4'):
                body += '<h4>' + line.text + '</h4>'
            elif str(line).__contains__('h5'):
                body += '<h5>' + line.text + '</h5>'
            elif str(line).__contains__('h6'):
                body += '<h6>' + line.text + '</h6>'    
            elif str(line).__contains__('blockquote'):
                if str(line).__contains__('type1'):
                    body += '<br><center><h3><font color="grey">' + line.text + '</font></h3></center><br>'
                elif str(line).__contains__('type2'):
                    line_text = re.sub(r"(<blockquote[^>]*>|</blockquote>)", '', str(line)).                    replace('</br>', '<br>')
                    body += '<br><div style="border-color:#acacac;border-width:0 0 0 2px;border-style:solid;padding:1px 0 0 12px;color:#666;line-height:18pt;text-align:left}"><font color="grey">' + line_text + '</font></div><br>'
                elif str(line).__contains__('type3'):
                    line_text = re.sub(r"(<blockquote[^>]*>|</blockquote>)", '', str(line)).                    replace('</br>', '<br>')
                    body += '<br><div style="border:1px solid #d7d7d7;text-align:left;padding:21px 25px 20px;color:#666;line-height:18pt}"><font color="grey">' + line_text + '</font></div><br>'
                else:
                    body += '<br><font color="grey">' + line.text + '</font><br>'
            elif str(line).__contains__('bullet'):
                body += '<li>' + line.text + '</li>'
            else:
                line_text = re.sub(r'(<blockquote[^>]*>|</blockquote>|<p[^>]*>|</p>)', '', str(line)).                    replace('<br/>', '<br>').replace('<b></b>', '')
                if line_text != '<br>':
                    #print(line_text)
                    body += '<br>' + line_text + '<br>'
            
    # 전체 html 파일 생성
    pad_front = '<html><head><style>' + '</style></head><body>'
    pad_rear = '</body></html>'
    html = pad_front + body + pad_rear

    return(html)

### urls in excel

In [22]:
excel_path = './url_samples.xlsx'
raw = pd.read_excel(excel_path, engine='openpyxl', header=None)

In [33]:
df = pd.DataFrame(raw.loc[:, 0].str.split('@', 1).tolist(), columns = ['row', 'id&order'])

In [36]:
id_and_order = pd.DataFrame(df['id&order'].str.split('/', 1).tolist(), columns = ['id', 'order'])

### import html files using the urls

In [50]:
base_url = 'https://brunch.co.kr/'
section_name = '휴가'
section_dir = os.path.join(os.getcwd(), 'scraped', section_name)

In [51]:
if not os.path.exists(section_dir):
    os.makedirs(section_dir)

In [55]:
for idx, row in id_and_order.iterrows():
    user_id = row['id']
    page_order = row['order']
    url = base_url + '@' + user_id + '/' + page_order
    
    html = brunch_to_html(url)
    
    save_path = os.path.join(section_dir, user_id + '-' + page_order + '.html')
    
    with open(save_path, 'w') as f:
        f.write(html)

* test

In [73]:
test=codecs.open('./scraped/휴가/7phs-11.html', 'r')
tmp = test.read()
print(tmp)

<html><head><style></style></head><body><h1>휴가의 의미</h1><br><h3>올해 첫 휴가를 사용한 후의 기록 </h3><br><h3>들어가며</h3><h4>며칠 전 올해 첫 '휴가'를 사용했습니다. 2021년이 시작되고 절반이 가까워지고 있는 시점에 첫 휴가를 사용했는데요, 오랜만에 휴가를 사용한 만큼 제겐 소중했고 여러 의미가 있었습니다. </h4><h4>저는 특별히 여행을 가거나, 큰 계획이 없어도 가끔씩 휴가를 사용하며 재충전 시간을 가지는 것을 선호하는 편이었어요. 작년까지는 한 달에 한 번은 휴가를 사용하는 것이 제게 큰 행복이었는데요, 올 해는 과거와는 다르게 휴가를 잘 사용하지 못하고 늦게 사용하게 되었습니다. </h4><h4>휴가를 사용 한 뒤 '휴가'에 대해서 제 생각을 정리해보았고, 여러분께 들려드릴까 해요. 읽으시면서 여러분의 휴가는 어떠신지 정리해보시고 저에게도 이야기 들려주세요. </h4><center><img src="http://t1.daumcdn.net/brunch/service/user/5sOI/image/XTrqL_UUwHKGmHEAbgn9xoiZ2dY.jpeg"><br></center><br><h3>1. 업무가 늘었어요(신규 업무 분장)  </h3><h4></h4><h4> 저는 기본적으로 '휴가'와 '일'은 적절하게 균형을 맞추면서 살아가는 것을 선호해요. "열심히 일한 당신, 떠나라!"라는 말도 좋아하는 편입니다. </h4><h4> 올해는 과거와는 조금 다른 균형을 맞추며 지내고 있어요. 먼저 작년보다 맡은 업무가 늘어나게 되었고, 업무 중에는 처음 해보게 되는 업무도 있었습니다. 기존에 제가 맡고 있던 MD 업무는 월말, 월초가 마감과 정산으로 바빴고 중순에는 휴가를 쓸 수 있었던 반면에 올해는 MD 업무와 함께 조직의 손익 및 경영관리 업무도 함께 되어 중순에도 자리를 비울 수 없게 되었습니다. 정리하면, 월말과 월초는 원래 바빴고 어쩌다 보니 올해는 중간에도 바쁘게 되었네요. </h

### cleaning (remove html tags)

In [67]:
def remove_tag(content):
   cleanr =re.compile('<.*?>')
   cleantext = re.sub(cleanr, '', content)
   return cleantext

In [89]:
save_path = './scraped/휴가'

In [94]:
for filename in os.listdir(save_path):
    if filename.endswith(".html"): 
        f = codecs.open(os.path.join(save_path, filename), 'r')
        raw_html = f.read()
        f.close()
        
        clean_html = remove_tag(raw_html)
        with open(os.path.join(save_path, filename)[:-4] + "txt", "w") as text_file:
            text_file.write(clean_html)
    else:
        continue

* test

In [75]:
print(remove_tag(tmp))

휴가의 의미올해 첫 휴가를 사용한 후의 기록 들어가며며칠 전 올해 첫 '휴가'를 사용했습니다. 2021년이 시작되고 절반이 가까워지고 있는 시점에 첫 휴가를 사용했는데요, 오랜만에 휴가를 사용한 만큼 제겐 소중했고 여러 의미가 있었습니다. 저는 특별히 여행을 가거나, 큰 계획이 없어도 가끔씩 휴가를 사용하며 재충전 시간을 가지는 것을 선호하는 편이었어요. 작년까지는 한 달에 한 번은 휴가를 사용하는 것이 제게 큰 행복이었는데요, 올 해는 과거와는 다르게 휴가를 잘 사용하지 못하고 늦게 사용하게 되었습니다. 휴가를 사용 한 뒤 '휴가'에 대해서 제 생각을 정리해보았고, 여러분께 들려드릴까 해요. 읽으시면서 여러분의 휴가는 어떠신지 정리해보시고 저에게도 이야기 들려주세요. 1. 업무가 늘었어요(신규 업무 분장)   저는 기본적으로 '휴가'와 '일'은 적절하게 균형을 맞추면서 살아가는 것을 선호해요. "열심히 일한 당신, 떠나라!"라는 말도 좋아하는 편입니다.  올해는 과거와는 조금 다른 균형을 맞추며 지내고 있어요. 먼저 작년보다 맡은 업무가 늘어나게 되었고, 업무 중에는 처음 해보게 되는 업무도 있었습니다. 기존에 제가 맡고 있던 MD 업무는 월말, 월초가 마감과 정산으로 바빴고 중순에는 휴가를 쓸 수 있었던 반면에 올해는 MD 업무와 함께 조직의 손익 및 경영관리 업무도 함께 되어 중순에도 자리를 비울 수 없게 되었습니다. 정리하면, 월말과 월초는 원래 바빴고 어쩌다 보니 올해는 중간에도 바쁘게 되었네요. 처음에는 새로운 업무를 진행하다 보니 휴가를 쓸 겨를도 없이 마무리하면 한 달이 지나가 있고, 분기도 지나가버렸습니다. 신규 업무의 경우에는 여러 동료들의 확인과 협의가 중요한 업무이기 때문에 동료들의 스케줄을 고려하게 되었고 휴가 사용은 점점 미뤄졌습니다. 또한 물어보고, 배워야 하는 부분들이 많아 쉽게 '휴가'를 사용하기 어려웠습니다.  2. 하다 보니 재미있더라     2021년 1분기는 신규업무에 정신없이 적응하니 끝나 있었고, 2분기가 되니 업무가 